# OpenCV Detection and Tracking
Based on: https://pysource.com/2021/01/28/object-tracking-with-opencv-and-python/

Object Detection - OpenCV
- Will be applied to each and every frame

Object Tracking - EuclidanDistTracker: 
- Will track the history of where each object was during time
- Using EuclidianDistTracker to identify the same object within frames

In [1]:
import cv2
import sys
from tracker import *

# Create tracker object
tracker = EuclideanDistTracker()

cap = cv2.VideoCapture("highway.mp4")
#cap = cv2.VideoCapture(0) # for using WebCam

# Read first frame
ok, frame = cap.read()
if not ok:
    print ('Cannot read video file')
    sys.exit()

# Extract Region of interest ROI
roi_coordinates = cv2.selectROI(frame, False)
cv2.destroyAllWindows()
print("Region of interest: ", roi_coordinates)

# Object detection from Stable camera, extract the moving objects from the stable camera¨
# the longer the history the more stable the object detection is
# the higher the value less detection, but also less false positives
object_detector = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=40)

while True:
    ret, frame = cap.read()
    height, width, _ = frame.shape    

    # 1. Object Detection
    roi = frame[int(roi_coordinates[1]):int(roi_coordinates[1] + roi_coordinates[3]), int(roi_coordinates[0]):int(roi_coordinates[0] + roi_coordinates[2])]
    mask = object_detector.apply(roi)

    # remove noise from the mask
    _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)

    # extract coordinates of moving objects
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # collect all detections (bounding boxes)
    detections = []
    for cnt in contours:
        # Calculate area and remove small elements
        area = cv2.contourArea(cnt)
        if area > 100:
            x, y, w, h = cv2.boundingRect(cnt)
            detections.append([x, y, w, h])

    # 2. Object Tracking
    boxes_ids = tracker.update(detections)
    for box_id in boxes_ids:
        x, y, w, h, id, direction = box_id
        cv2.putText(roi, str(id) + " " + format(direction,".2f"), (x, y - 15), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2)
        cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 3)

    cv2.imshow("roi", roi)
    cv2.imshow("Frame", frame)
    cv2.imshow("Mask", mask)

    key = cv2.waitKey(0)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

Region of interest:  (556, 434, 258, 112)
